# wczytywanie

In [1]:
from parse import parse_task_1_2

In [2]:
meta, sentences = parse_task_1_2('training/file_39.ttl')

In [3]:
sent = list(sentences.keys())[0]

In [4]:
sent

'Knowledge Services and Dotstaff is incredibly happy to be a part of this wonderful community, said Joe Bielawski, the president of Knowledge Services.'

# parsowanie

In [5]:
from entities_recognition_tester import process_using_spacy
import en_core_web_sm

In [6]:
pary = process_using_spacy(sent, en_core = en_core_web_sm.load(), format_result=True)
pary

[('Dotstaff', 'ORG'),
 ('Joe Bielawski', 'PERSON'),
 ('Knowledge Services', 'ORG')]

In [15]:
import itertools
perm = list(itertools.permutations(pary[1:], 2))
perm

[(('Joe Bielawski', 'PERSON'), ('Knowledge Services', 'ORG')),
 (('Knowledge Services', 'ORG'), ('Joe Bielawski', 'PERSON'))]

In [8]:
big_dict = {
     ('Organisation', 'Organisation'): ['affiliation'],
     ('Person', 'EducationalInstitution'): ['almaMater'],
     ('Band', 'Person'): ['formerBandMember', 'bandMember'],
     ('Person', 'Place'): ['deathPlace', 'birthPlace'],
     ('Organisation', 'Person'): ['president', 'ceo'],
     ('Person', 'Person'): ['spouse', 'relative', 'parent', 'child'],
     ('Athlete', 'SportsTeam'): ['formerTeam', 'debutTeam', 'club'],
     ('Organisation', 'Country'): ['country'],
     ('Person', 'Country'): ['country'],
     ('Place', 'Country'): ['country'],
     ('PopulatedPlace', 'PopulatedPlace'): ['department'],
     ('Place', 'PopulatedPlace'): ['district'],
     ('Scientist', 'Person'): ['doctoralStudent', 'doctoralAdvisor'],
     ('Person', 'Organisation'): ['employer'],
     ('Organisation', 'City'): ['foundationPlace'],
     ('Organisation', 'PopulatedPlace'): ['headquarter'],
     ('Organisation', 'Settlement'): ['hometown'],
     ('Person', 'Settlement'): ['hometown'],
     ('PopulatedPlace', 'Person'): ['leaderName'],
     ('Place', 'Place'): ['locatedInArea'],
     ('Organisation', 'Place'): ['location'],
     ('Person', 'Place'): ['location'],
     ('Place', 'Place'): ['location'],
     ('Person', 'Country'): ['nationality'],
     ('Company', 'Company'): ['subsidiary'],
     ('ArchitecturalStructure', 'Organisation'): ['tenant'],
     ('Athlete', 'Person'): ['trainer']
}

In [9]:
maps = {
    'ORG': 'Organisation',
    'PERSON': 'Person'
}

In [16]:
rel = []
for a,b in perm:
    co = big_dict[(maps[a[1]], maps[b[1]])]
    for c in co:
        if c in sent:
            rel.append((a, b, c)) 
            
rel

[(('Knowledge Services', 'ORG'), ('Joe Bielawski', 'PERSON'), 'president')]

# odpytywania

In [11]:
from sparql_resource_property_getter import get_resource, find_property

(('Knowledge Services', 'ORG'), ('Joe Bielawski', 'PERSON'), 'president')

In [12]:
for a, b, r in rel:
    print(a[0], ': ', get_resource(a[0]))
    print(b[0], ': ', get_resource(b[0]))
    print(r, ': ', get_resource(r))
    print()

Dotstaff :  []
Joe Bielawski :  []
president :  []

Knowledge Services :  []
Joe Bielawski :  []
president :  []



In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# zwracanie

In [26]:
to = sentences[sent].split('\n')
to

['<https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=0,150>',
 '        a               nif:Context , nif:String , nif:RFC5147String ;',
 '        nif:beginIndex  "0"^^xsd:nonNegativeInteger ;',
 '        nif:endIndex    "150"^^xsd:nonNegativeInteger ;',
 '        nif:isString    "Knowledge Services and Dotstaff is incredibly happy to be a part of this wonderful community, said Joe Bielawski, the president of Knowledge Services." .']

In [62]:
word = rel[0][0][0]
start_index = sent.find(word)
end_index = start_index+len(word)

word_ns = word.replace(' ', '_')

print(to[0].split('#')[0]+f'#char={start_index},{end_index}> ;')
print(" ".join(to[1].split()).replace('nif:Context', 'nif:Phrase'))
print(f'nif:anchorOf  "{word}"^^xsd:string ;')
print(f'nif:beginIndex  "{start_index}"^^xsd:nonNegativeInteger ;')
print(f'nif:endIndex  "{end_index}"^^xsd:nonNegativeInteger ;')
print(to[0] + ' ;')
if len(get_resource(word_ns)) > 0:
    print(f'itsrdf:taIdentRef  dbr:{word_ns} .')
else:
    print(f'http://aksw.org/notInWiki/{word_ns} .')

<https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=0,18> ;
a nif:Phrase , nif:String , nif:RFC5147String ;
nif:anchorOf  "Knowledge Services"^^xsd:string ;
nif:beginIndex  "0"^^xsd:nonNegativeInteger ;
nif:endIndex  "18"^^xsd:nonNegativeInteger ;
<https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=0,150> ;
http://aksw.org/notInWiki/Knowledge_Services .


In [63]:
word = rel[0][1][0]
start_index = sent.find(word)
end_index = start_index+len(word)

word_ns = word.replace(' ', '_')

print(to[0].split('#')[0]+f'#char={start_index},{end_index}> ;')
print(" ".join(to[1].split()).replace('nif:Context', 'nif:Phrase'))
print(f'nif:anchorOf  "{word}"^^xsd:string ;')
print(f'nif:beginIndex  "{start_index}"^^xsd:nonNegativeInteger ;')
print(f'nif:endIndex  "{end_index}"^^xsd:nonNegativeInteger ;')
print(to[0] + ' ;')
if len(get_resource(word_ns)) > 0:
    print(f'itsrdf:taIdentRef  dbr:{word_ns} .')
else:
    print(f'http://aksw.org/notInWiki/{word_ns} .')

<https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=99,112> ;
a nif:Phrase , nif:String , nif:RFC5147String ;
nif:anchorOf  "Joe Bielawski"^^xsd:string ;
nif:beginIndex  "99"^^xsd:nonNegativeInteger ;
nif:endIndex  "112"^^xsd:nonNegativeInteger ;
<https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=0,150> ;
http://aksw.org/notInWiki/Joe_Bielawski .


In [17]:
[]
  a  rdf:Statement , oa:Annotation ;
  rdf:object  dbr:Conor_McGregor ;
  rdf:predicate  dbo:trainer ;
  rdf:subject  aksw:John_Kavanagh ;
  oa:hasTarget [
    a oa:SpecificResource;
    oa:hasSource <http://www.ontologydesignpatterns.org/data/oke-challenge-2018/task-3/sentence-1#char=0,78> ] .
"""

'\n<http://www.ontologydesignpatterns.org/data/oke-challenge-2018/task-3/sentence-1#char=0,22>\n  a  nif:RFC5147String , nif:String , nif:Phrase ;\n  nif:anchorOf  "Conor McGregor\'s"^^xsd:string ;\n  nif:beginIndex  "0"^^xsd:nonNegativeInteger ;\n  nif:endIndex  "22"^^xsd:nonNegativeInteger ;\n  nif:referenceContext  <http://www.ontologydesignpatterns.org/data/oke-challenge-2018/task-3/sentence-1#char=0,78> ;\n  itsrdf:taIdentRef  dbr:Conor_McGregor .\n  \n<http://www.ontologydesignpatterns.org/data/oke-challenge-2018/task-3/sentence-1#char=35,48>\n  a  nif:RFC5147String , nif:String , nif:Phrase ;\n  nif:anchorOf  "John Kavanagh"^^xsd:string ;\n  nif:beginIndex  "35"^^xsd:nonNegativeInteger ;\n  nif:endIndex  "48"^^xsd:nonNegativeInteger ;\n  nif:referenceContext  <http://www.ontologydesignpatterns.org/data/oke-challenge-2018/task-3/sentence-1#char=0,78> ;\n  itsrdf:taIdentRef  aksw:John_Kavanagh . \n\n[]\n  a  rdf:Statement , oa:Annotation ;\n  rdf:object  dbr:Conor_McGregor ;\n  rd